In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from warnings import filterwarnings
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint

In [ ]:
x_train = []
y_train = []
image_size = 128

folderPath = os.path.join('drive/MyDrive/x-ray/normal')
for j in os.listdir(folderPath):
    img = cv2.imread(os.path.join(folderPath,j))
    img = cv2.resize(img,(image_size, image_size))
    x_train.append(img)
    y_train.append(0)
    # if len(y_train) == 1000:
    #     break

folderPath = os.path.join('drive/MyDrive/x-ray/bad')
for j in os.listdir(folderPath):
    img = cv2.imread(os.path.join(folderPath,j))
    img = cv2.resize(img,(image_size, image_size))
    x_train.append(img)
    y_train.append(1)
    # if len(y_train) == 2000:
    #     break


x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train, y_train = shuffle(x_train, y_train, random_state=127)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
                                      test_size=0.1, random_state=127)

In [ ]:
from tensorflow.keras.applications import EfficientNetB1

effnet = EfficientNetB1(weights='imagenet',
                        include_top=False,
                        input_shape=(128,128,3))

Мы тестируем именно модели EfficientNet B0/B1, потому что B2-B7 рассчитаны на более тяжелые задачи анализа фотографий.

In [ ]:
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dense(128,activation='relu')(model)
model = tf.keras.layers.Dropout(rate=0.4)(model)
model = tf.keras.layers.Dense(1,activation='sigmoid')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer = 'Adam',
              metrics= ['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 128, 128, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 128, 128, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 128, 128, 3)          0         ['normalization[0][0]']   

In [ ]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("drive/MyDrive/x-ray/x_ray_effnet_b1.h5",
                             monitor="val_accuracy",
                             save_best_only=True,
                             mode="auto",
                             verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                              factor = 0.3,
                              patience = 2,
                              min_delta = 0.0025,
                              mode='auto',
                              verbose=1)

In [ ]:
history = model.fit(x_train, y_train, validation_split=0.1, epochs = 16, verbose=1, batch_size=32,
                   callbacks=[tensorboard, checkpoint, reduce_lr])

Epoch 1/16
236/236 [==============================] - ETA: 0s - loss: 0.1177 - accuracy: 0.9609
Epoch 1: val_accuracy improved from -inf to 0.94994, saving model to drive/MyDrive/x-ray/x_ray_effnet_b1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


236/236 [==============================] - 869s 4s/step - loss: 0.1177 - accuracy: 0.9609 - val_loss: 0.1791 - val_accuracy: 0.9499 - lr: 0.0010
Epoch 2/16
236/236 [==============================] - ETA: 0s - loss: 0.0652 - accuracy: 0.9768
Epoch 2: val_accuracy improved from 0.94994 to 0.98212, saving model to drive/MyDrive/x-ray/x_ray_effnet_b1.h5
236/236 [==============================] - 885s 4s/step - loss: 0.0652 - accuracy: 0.9768 - val_loss: 0.0462 - val_accuracy: 0.9821 - lr: 0.0010
Epoch 3/16
236/236 [==============================] - ETA: 0s - loss: 0.0478 - accuracy: 0.9822
Epoch 3: val_accuracy did not improve from 0.98212
236/236 [==============================] - 881s 4s/step - loss: 0.0478 - accuracy: 0.9822 - val_loss: 0.0696 - val_accuracy: 0.9762 - lr: 0.0010
Epoch 4/16
236/236 [==============================] - ETA: 0s - loss: 0.0487 - accuracy: 0.9817
Epoch 4: val_accuracy did not improve from 0.98212

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00030000

KeyboardInterrupt: 

In [ ]:
model.save("drive/MyDrive/x-ray/x_ray_effnet_b1_last.h5")
model.save("drive/MyDrive/x-ray/x_ray_effnet_b1_last.keras")

In [17]:
pred = model.predict(x_test)

30/30 [==============================] - 28s 794ms/step


In [18]:
print(classification_report(y_test,pred.round()))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       438
           1       0.99      0.98      0.99       494

    accuracy                           0.99       932
   macro avg       0.99      0.99      0.99       932
weighted avg       0.99      0.99      0.99       932



In [ ]:
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_red = ["#331313", "#582626", '#9E1717', '#D35151', '#E9B4B4']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']

In [ ]:
filterwarnings('ignore')

epochs = [i for i in range(7)]
fig, ax = plt.subplots(1,2,figsize=(14,7))
train_acc = history['accuracy']
train_loss = history['loss']
val_acc = history['val_accuracy']
val_loss = history['val_loss']

fig.text(s='Epochs vs. Training and Validation Accuracy/Loss',size=18,fontweight='bold',
             fontname='monospace',color=colors_dark[1],y=1,x=0.28,alpha=0.8)

sns.despine()
ax[0].plot(epochs, train_acc, marker='o',markerfacecolor=colors_green[2],color=colors_green[3],
           label = 'Training Accuracy')
ax[0].plot(epochs, val_acc, marker='o',markerfacecolor=colors_red[2],color=colors_red[3],
           label = 'Validation Accuracy')
ax[0].legend(frameon=False)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')

sns.despine()
ax[1].plot(epochs, train_loss, marker='o',markerfacecolor=colors_green[2],color=colors_green[3],
           label ='Training Loss')
ax[1].plot(epochs, val_loss, marker='o',markerfacecolor=colors_red[2],color=colors_red[3],
           label = 'Validation Loss')
ax[1].legend(frameon=False)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Training & Validation Loss')

fig.show()